Using cycleGAN model to harmonize the images. The model is making the scans from Guys look more like the scans from HH.

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install monai nibabel numpy scipy matplotlib tqdm SimpleITK
!pip install neuroHarmonize nilearn tqdm
!pip install neuroHarmonize neuroCombat

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 137.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neuroCombat: filename=neuroCombat-0.2.12-py3-none-any.whl size=6353 sha256=2ce8d7b31d69ca1289f8f256c48024c8c4af33faccbc764ceccb30028d9f75f3
  Stored in directory: /root/.cache/pip/wheels/be/6a/95/9d827c0f3cc23854b5fbd00fbc8a052d492538dc16bd20f7a2
Successfully built neuroCombat


In [ ]:
import torch
import nibabel as nib
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
from nilearn import datasets, image, masking
from tqdm import tqdm
from neuroHarmonize import harmonizationLearn
import seaborn as sns
import imageio
import tensorflow as tf
from glob import glob
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Training data
HH_dir = "/content/drive/MyDrive/finalT1T2Data/T2_train_by_loc/HH/"
guys_dir = "/content/drive/MyDrive/finalT1T2Data/T2_train_by_loc/Guys/"
train_dir = "/content/drive/MyDrive/finalT1T2Data/train_w_loc/T2"


train_prediction_files = sorted([f for f in os.listdir(train_dir)])
hh_train_predictions_names = []
guys_train_predictions_names= []
for f in train_prediction_files:
  f_full_name = os.path.join(train_dir, f)
  img = Image.open(f_full_name)
  if f_full_name.split('_')[3] == 'HH':
    img.save(HH_dir + f)
  else:
    img.save(guys_dir + f)

In [ ]:
# Testing data
HH_dir = "/content/drive/MyDrive/finalT1T2Data/T2_test_by_loc/HH/"
guys_dir = "/content/drive/MyDrive/finalT1T2Data/T2_test_by_loc/Guys/"
test_dir = "/content/drive/MyDrive/finalT1T2Data/test_w_loc/T2"


test_prediction_files = sorted([f for f in os.listdir(test_dir)])
hh_test_predictions_names = []
guys_test_predictions_names= []
for f in test_prediction_files:
  f_full_name = os.path.join(test_dir, f)
  img = Image.open(f_full_name)
  if f_full_name.split('_')[3] == 'HH':
    img.save(HH_dir + f)
  else:
    img.save(guys_dir + f)

Train CycleGAN model to convert Guys T2 scans into the style of HH T2 scans.

In [ ]:
# CycleGAN

import os
import random
from glob import glob
from PIL import Image
import numpy as np
from tqdm import tqdm
import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# parameters
siteA_path = "/content/drive/MyDrive/finalT1T2Data/T2_train_by_loc/Guys/"   # source domain
siteB_path = "/content/drive/MyDrive/finalT1T2Data/T2_train_by_loc/HH/"   # target domain
out_dir = "/content/drive/MyDrive/finalT1T2Data/outputs/harmonized_Guys_train"
checkpoints_dir = "checkpoints/cyclegan"
sample_dir = "samples"
img_size = 256
batch_size = 4
epochs = 100
lr = 2e-4
beta1 = 0.5
lambda_cycle = 10.0
lambda_id = 5.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_every = 10
seed = 42

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
os.makedirs(out_dir, exist_ok=True)
os.makedirs(checkpoints_dir, exist_ok=True)
os.makedirs(sample_dir, exist_ok=True)

# Dataset
class PNGSliceDataset(Dataset):
    def __init__(self, folder, img_size=256):
        paths = sorted([p for p in glob(os.path.join(folder, "*.png")) if os.path.isfile(p)])
        if len(paths) == 0:
            raise RuntimeError(f"No PNGs found in {folder}")
        self.paths = paths
        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((img_size, img_size), interpolation=Image.BILINEAR),
            transforms.ToTensor(),
        ])
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        p = self.paths[idx]
        img = Image.open(p)
        img = self.transform(img)
        return img, p

dsA = PNGSliceDataset(siteA_path, img_size=img_size)
dsB = PNGSliceDataset(siteB_path, img_size=img_size)

loaderA = DataLoader(dsA, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
loaderB = DataLoader(dsB, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)

print("Num A:", len(dsA), "Num B:", len(dsB), "Device:", device)

#Utils
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

# Replay buffer for generated images
class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0
        self.max_size = max_size
        self.data = []
    def push_and_pop(self, data):
        to_return = []
        for element in data:
            element = element.unsqueeze(0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.random() > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    tmp = self.data[i].clone()
                    self.data[i] = element
                    to_return.append(tmp)
                else:
                    to_return.append(element)
        return torch.cat(to_return, dim=0)

# Save side-by-side samples
def save_sample(epoch, batch_idx, real_A, real_B, fake_B, fake_A, prefix="sample"):
    B = min(4, real_A.shape[0])
    import matplotlib.pyplot as plt
    fig, axes = plt.subplots(B, 4, figsize=(12, 3*B))
    for i in range(B):
        axes[i,0].imshow(real_A[i,0].cpu().numpy(), cmap='gray'); axes[i,0].set_title("A (src)")
        axes[i,1].imshow(fake_B[i,0].cpu().numpy(), cmap='gray'); axes[i,1].set_title("A -> B")
        axes[i,2].imshow(real_B[i,0].cpu().numpy(), cmap='gray'); axes[i,2].set_title("B (tgt)")
        axes[i,3].imshow(fake_A[i,0].cpu().numpy(), cmap='gray'); axes[i,3].set_title("B -> A")
        for j in range(4):
            axes[i,j].axis('off')
    plt.tight_layout()
    fname = os.path.join(sample_dir, f"{prefix}_epoch{epoch:03d}_batch{batch_idx}.png")
    plt.savefig(fname, bbox_inches='tight', dpi=150)
    plt.close(fig)

# ----------------- Models ----------------
# ResNet block used in generator
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3, padding=0),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),

            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3, padding=0),
            nn.InstanceNorm2d(dim),
        )
    def forward(self, x):
        return x + self.conv_block(x)

# Generator
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc=1, output_nc=1, ngf=64, n_blocks=6):
        super().__init__()
        model = []
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0),
                  nn.InstanceNorm2d(ngf),
                  nn.ReLU(True)]
        # downsample
        n_downsampling = 2
        mult = 1
        for i in range(n_downsampling):
            mult_prev = mult
            mult = mult * 2
            model += [nn.Conv2d(ngf * mult_prev, ngf * mult, kernel_size=3, stride=2, padding=1),
                      nn.InstanceNorm2d(ngf * mult),
                      nn.ReLU(True)]
        # resnet blocks
        for i in range(n_blocks):
            model += [ResnetBlock(ngf * mult)]
        # upsample
        for i in range(n_downsampling):
            mult_prev = mult
            mult = mult // 2
            model += [nn.ConvTranspose2d(ngf * mult_prev, ngf * mult, kernel_size=3, stride=2,
                                         padding=1, output_padding=1),
                      nn.InstanceNorm2d(ngf * mult),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0),
                  nn.Tanh()]
        self.model = nn.Sequential(*model)
    def forward(self, x):
        return self.model(x)

# PatchGAN discriminator
class NLayerDiscriminator(nn.Module):
    def __init__(self, input_nc=1, ndf=64, n_layers=3):
        super().__init__()
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, True)
        ]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=4, stride=2, padding=1),
                nn.InstanceNorm2d(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]
        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]
        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=4, stride=1, padding=1)]
        self.model = nn.Sequential(*sequence)
    def forward(self, x):
        return self.model(x)

# ----------------- Initialize models ----------------
G_AB = ResnetGenerator().to(device)
G_BA = ResnetGenerator().to(device)
D_A = NLayerDiscriminator().to(device)
D_B = NLayerDiscriminator().to(device)

G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)

# Losses
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizers
optimizer_G = torch.optim.Adam(list(G_AB.parameters()) + list(G_BA.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(beta1, 0.999))

# LR schedulers (linear decay after half epochs)
def lambda_rule(epoch):
    return 1.0 - max(0, epoch - epochs/2) / (epochs/2)
scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lambda_rule)
scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=lambda_rule)
scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=lambda_rule)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# target labels for LSGAN
def real_target_like(x):
    return torch.ones_like(x, device=device)
def fake_target_like(x):
    return torch.zeros_like(x, device=device)

# Training loop
print("Starting training...")
start_time = time.time()
for epoch in range(1, epochs+1):
    G_AB.train(); G_BA.train(); D_A.train(); D_B.train()
    loop = tqdm(zip(loaderA, loaderB), total=min(len(loaderA), len(loaderB)))
    recon_cycle_loss = 0.0
    gan_loss_val = 0.0
    id_loss_val = 0.0

    for i, ((real_A, pathA), (real_B, pathB)) in enumerate(loop):
        real_A = real_A.to(device) * 2.0 - 1.0
        real_B = real_B.to(device) * 2.0 - 1.0
        optimizer_G.zero_grad()

        same_B = G_AB(real_B)
        loss_id_B = criterion_identity(same_B, real_B) * lambda_cycle * lambda_id
        same_A = G_BA(real_A)
        loss_id_A = criterion_identity(same_A, real_A) * lambda_cycle * lambda_id

        # GAN loss
        fake_B = G_AB(real_A)
        pred_fake_B = D_B(fake_B)
        loss_GAN_AB = criterion_GAN(pred_fake_B, real_target_like(pred_fake_B))

        fake_A = G_BA(real_B)
        pred_fake_A = D_A(fake_A)
        loss_GAN_BA = criterion_GAN(pred_fake_A, real_target_like(pred_fake_A))

        # cycle loss
        rec_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(rec_A, real_A) * lambda_cycle
        rec_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(rec_B, real_B) * lambda_cycle

        # total generator loss
        loss_G = loss_GAN_AB + loss_GAN_BA + loss_cycle_A + loss_cycle_B + loss_id_A + loss_id_B
        loss_G.backward()
        optimizer_G.step()

        optimizer_D_B.zero_grad()
        # Real
        pred_real_B = D_B(real_B)
        loss_D_real_B = criterion_GAN(pred_real_B, real_target_like(pred_real_B))
        # Fake
        fake_B_detached = fake_B_buffer.push_and_pop(fake_B.detach())
        pred_fake_B = D_B(fake_B_detached)
        loss_D_fake_B = criterion_GAN(pred_fake_B, fake_target_like(pred_fake_B))
        loss_D_B = (loss_D_real_B + loss_D_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        optimizer_D_A.zero_grad()
        pred_real_A = D_A(real_A)
        loss_D_real_A = criterion_GAN(pred_real_A, real_target_like(pred_real_A))
        fake_A_detached = fake_A_buffer.push_and_pop(fake_A.detach())
        pred_fake_A = D_A(fake_A_detached)
        loss_D_fake_A = criterion_GAN(pred_fake_A, fake_target_like(pred_fake_A))
        loss_D_A = (loss_D_real_A + loss_D_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # logging
        recon_cycle_loss += (loss_cycle_A.item() + loss_cycle_B.item())
        gan_loss_val += (loss_GAN_AB.item() + loss_GAN_BA.item())
        id_loss_val += (loss_id_A.item() + loss_id_B.item())

        if i % 50 == 0:
            loop.set_description(f"Epoch[{epoch}/{epochs}] step {i} lossG {loss_G.item():.4f}")

    # update lr
    scheduler_G.step()
    scheduler_D_A.step()
    scheduler_D_B.step()

    # Save samples & checkpoint
    if epoch % save_every == 0 or epoch == 1 or epoch == epochs:
        with torch.no_grad():
            G_AB.eval(); G_BA.eval()
            real_A_sample, _ = next(iter(loaderA))
            real_B_sample, _ = next(iter(loaderB))
            real_A_sample = real_A_sample.to(device) * 2.0 - 1.0
            real_B_sample = real_B_sample.to(device) * 2.0 - 1.0
            fake_B_sample = G_AB(real_A_sample).cpu() * 0.5 + 0.5
            fake_A_sample = G_BA(real_B_sample).cpu() * 0.5 + 0.5
            real_A_vis = (real_A_sample.cpu() * 0.5 + 0.5)
            real_B_vis = (real_B_sample.cpu() * 0.5 + 0.5)
            save_sample(epoch, 0, real_A_vis, real_B_vis, fake_B_sample, fake_A_sample, prefix="epoch")
        torch.save({
            'G_AB': G_AB.state_dict(),
            'G_BA': G_BA.state_dict(),
            'D_A': D_A.state_dict(),
            'D_B': D_B.state_dict(),
            'optimizer_G': optimizer_G.state_dict(),
            'optimizer_D_A': optimizer_D_A.state_dict(),
            'optimizer_D_B': optimizer_D_B.state_dict(),
        }, os.path.join(checkpoints_dir, f"cyclegan_epoch{epoch:03d}.pth"))
        print(f"[Epoch {epoch}] Saved samples and checkpoint.")


    print(f"Epoch {epoch} summary: cycle_loss {recon_cycle_loss/len(loaderA):.6f}, gan_loss {gan_loss_val/len(loaderA):.6f}, id_loss {id_loss_val/len(loaderA):.6f}")

elapsed = time.time() - start_time
print("Training finished in {:.1f} s".format(elapsed))


# Save final trained models
torch.save({
    'G_AB': G_AB.state_dict(),
    'G_BA': G_BA.state_dict(),
    'D_A': D_A.state_dict(),
    'D_B': D_B.state_dict(),
    'optimizer_G': optimizer_G.state_dict(),
    'optimizer_D_A': optimizer_D_A.state_dict(),
    'optimizer_D_B': optimizer_D_B.state_dict(),
    'epochs': epochs,
}, os.path.join(checkpoints_dir, "cyclegan_final_T2.pth"))

print("Final model saved to:", os.path.join(checkpoints_dir, "cyclegan_final_T2.pth"))
# Harmonize Guys Images
print("Generating harmonized outputs for Site A ...")
G_AB.eval()
os.makedirs(out_dir, exist_ok=True)
with torch.no_grad():
    for i in range(len(dsA)):
        x, p = dsA[i]
        x_t = x.unsqueeze(0).to(device) * 2.0 - 1.0
        y = G_AB(x_t)              # in [-1,1]
        y = (y.cpu().squeeze(0).squeeze(0).numpy() * 0.5 + 0.5)  # to [0,1]
        y = (np.clip(y, 0.0, 1.0) * 255.0).astype(np.uint8)
        fname = os.path.basename(p)
        Image.fromarray(y).save(os.path.join(out_dir, fname))
print("Harmonized images saved to:", out_dir)



Num A: 88 Num B: 88 Device: cuda
Starting training...


Epoch[1/100] step 0 lossG 91.6746: 100%|██████████| 22/22 [00:35<00:00,  1.60s/it]


[Epoch 1] Saved samples and checkpoint.
Epoch 1 summary: cycle_loss 3.586406, gan_loss 2.814836, id_loss 16.389591


Epoch[2/100] step 0 lossG 9.5419: 100%|██████████| 22/22 [00:34<00:00,  1.55s/it]

Epoch 2 summary: cycle_loss 1.390958, gan_loss 0.813554, id_loss 6.209581


...

Epoch 98 summary: cycle_loss 0.434428, gan_loss 0.717795, id_loss 1.306365



Epoch[99/100] step 0 lossG 2.6251: 100%|██████████| 22/22 [00:34<00:00,  1.55s/it]

Epoch 99 summary: cycle_loss 0.434293, gan_loss 0.718662, id_loss 1.284859



Epoch[100/100] step 0 lossG 2.4592: 100%|██████████| 22/22 [00:34<00:00,  1.55s/it]


[Epoch 100] Saved samples and checkpoint.
Epoch 100 summary: cycle_loss 0.428461, gan_loss 0.712813, id_loss 1.270243
Training finished in 3460.8 s
Final model saved to: checkpoints/cyclegan/cyclegan_final_T2.pth
Generating harmonized outputs for Site A ...
Harmonized images saved to: /content/drive/MyDrive/finalT1T2Data/outputs/harmonized_Guys_train


Train CycleGAN model to convert Guys T1 scans into the style of HH T1 scans.

In [ ]:
# CycleGAN

import os
import random
from glob import glob
from PIL import Image
import numpy as np
from tqdm import tqdm
import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# parameters
siteA_path = "/content/drive/MyDrive/finalT1T2Data/T1_train_by_loc/Guys/"   # source domain
siteB_path = "/content/drive/MyDrive/finalT1T2Data/T1_train_by_loc/HH/"   # target domain
out_dir = "/content/drive/MyDrive/finalT1T2Data/outputs/harmonized_Guys_train_T1"
checkpoints_dir = "checkpoints/cyclegan"
sample_dir = "samples"
img_size = 256
batch_size = 4
epochs = 100
lr = 2e-4
beta1 = 0.5
lambda_cycle = 10.0
lambda_id = 5.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_every = 10
seed = 42

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
os.makedirs(out_dir, exist_ok=True)
os.makedirs(checkpoints_dir, exist_ok=True)
os.makedirs(sample_dir, exist_ok=True)

# Dataset
class PNGSliceDataset(Dataset):
    def __init__(self, folder, img_size=256):
        paths = sorted([p for p in glob(os.path.join(folder, "*.png")) if os.path.isfile(p)])
        if len(paths) == 0:
            raise RuntimeError(f"No PNGs found in {folder}")
        self.paths = paths
        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((img_size, img_size), interpolation=Image.BILINEAR),
            transforms.ToTensor(),
        ])
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        p = self.paths[idx]
        img = Image.open(p)
        img = self.transform(img)
        return img, p

dsA = PNGSliceDataset(siteA_path, img_size=img_size)
dsB = PNGSliceDataset(siteB_path, img_size=img_size)

loaderA = DataLoader(dsA, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
loaderB = DataLoader(dsB, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)

print("Num A:", len(dsA), "Num B:", len(dsB), "Device:", device)

#Utils
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

# Replay buffer for generated images
class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0
        self.max_size = max_size
        self.data = []
    def push_and_pop(self, data):
        to_return = []
        for element in data:
            element = element.unsqueeze(0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.random() > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    tmp = self.data[i].clone()
                    self.data[i] = element
                    to_return.append(tmp)
                else:
                    to_return.append(element)
        return torch.cat(to_return, dim=0)

# Save side-by-side samples
def save_sample(epoch, batch_idx, real_A, real_B, fake_B, fake_A, prefix="sample"):
    B = min(4, real_A.shape[0])
    import matplotlib.pyplot as plt
    fig, axes = plt.subplots(B, 4, figsize=(12, 3*B))
    for i in range(B):
        axes[i,0].imshow(real_A[i,0].cpu().numpy(), cmap='gray'); axes[i,0].set_title("A (src)")
        axes[i,1].imshow(fake_B[i,0].cpu().numpy(), cmap='gray'); axes[i,1].set_title("A -> B")
        axes[i,2].imshow(real_B[i,0].cpu().numpy(), cmap='gray'); axes[i,2].set_title("B (tgt)")
        axes[i,3].imshow(fake_A[i,0].cpu().numpy(), cmap='gray'); axes[i,3].set_title("B -> A")
        for j in range(4):
            axes[i,j].axis('off')
    plt.tight_layout()
    fname = os.path.join(sample_dir, f"{prefix}_epoch{epoch:03d}_batch{batch_idx}.png")
    plt.savefig(fname, bbox_inches='tight', dpi=150)
    plt.close(fig)

# ----------------- Models ----------------
# ResNet block used in generator
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3, padding=0),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),

            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3, padding=0),
            nn.InstanceNorm2d(dim),
        )
    def forward(self, x):
        return x + self.conv_block(x)

# Generator
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc=1, output_nc=1, ngf=64, n_blocks=6):
        super().__init__()
        model = []
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0),
                  nn.InstanceNorm2d(ngf),
                  nn.ReLU(True)]
        # downsample
        n_downsampling = 2
        mult = 1
        for i in range(n_downsampling):
            mult_prev = mult
            mult = mult * 2
            model += [nn.Conv2d(ngf * mult_prev, ngf * mult, kernel_size=3, stride=2, padding=1),
                      nn.InstanceNorm2d(ngf * mult),
                      nn.ReLU(True)]
        # resnet blocks
        for i in range(n_blocks):
            model += [ResnetBlock(ngf * mult)]
        # upsample
        for i in range(n_downsampling):
            mult_prev = mult
            mult = mult // 2
            model += [nn.ConvTranspose2d(ngf * mult_prev, ngf * mult, kernel_size=3, stride=2,
                                         padding=1, output_padding=1),
                      nn.InstanceNorm2d(ngf * mult),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0),
                  nn.Tanh()]
        self.model = nn.Sequential(*model)
    def forward(self, x):
        return self.model(x)

# PatchGAN discriminator
class NLayerDiscriminator(nn.Module):
    def __init__(self, input_nc=1, ndf=64, n_layers=3):
        super().__init__()
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, True)
        ]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=4, stride=2, padding=1),
                nn.InstanceNorm2d(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]
        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]
        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=4, stride=1, padding=1)]
        self.model = nn.Sequential(*sequence)
    def forward(self, x):
        return self.model(x)

# ----------------- Initialize models ----------------
G_AB = ResnetGenerator().to(device)
G_BA = ResnetGenerator().to(device)
D_A = NLayerDiscriminator().to(device)
D_B = NLayerDiscriminator().to(device)

G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)

# Losses
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizers
optimizer_G = torch.optim.Adam(list(G_AB.parameters()) + list(G_BA.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(beta1, 0.999))

# LR schedulers (linear decay after half epochs)
def lambda_rule(epoch):
    return 1.0 - max(0, epoch - epochs/2) / (epochs/2)
scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lambda_rule)
scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=lambda_rule)
scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=lambda_rule)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# target labels for LSGAN
def real_target_like(x):
    return torch.ones_like(x, device=device)
def fake_target_like(x):
    return torch.zeros_like(x, device=device)

# Training loop
print("Starting training...")
start_time = time.time()
for epoch in range(1, epochs+1):
    G_AB.train(); G_BA.train(); D_A.train(); D_B.train()
    loop = tqdm(zip(loaderA, loaderB), total=min(len(loaderA), len(loaderB)))
    recon_cycle_loss = 0.0
    gan_loss_val = 0.0
    id_loss_val = 0.0

    for i, ((real_A, pathA), (real_B, pathB)) in enumerate(loop):
        real_A = real_A.to(device) * 2.0 - 1.0
        real_B = real_B.to(device) * 2.0 - 1.0
        optimizer_G.zero_grad()

        same_B = G_AB(real_B)
        loss_id_B = criterion_identity(same_B, real_B) * lambda_cycle * lambda_id
        same_A = G_BA(real_A)
        loss_id_A = criterion_identity(same_A, real_A) * lambda_cycle * lambda_id

        # GAN loss
        fake_B = G_AB(real_A)
        pred_fake_B = D_B(fake_B)
        loss_GAN_AB = criterion_GAN(pred_fake_B, real_target_like(pred_fake_B))

        fake_A = G_BA(real_B)
        pred_fake_A = D_A(fake_A)
        loss_GAN_BA = criterion_GAN(pred_fake_A, real_target_like(pred_fake_A))

        # cycle loss
        rec_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(rec_A, real_A) * lambda_cycle
        rec_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(rec_B, real_B) * lambda_cycle

        # total generator loss
        loss_G = loss_GAN_AB + loss_GAN_BA + loss_cycle_A + loss_cycle_B + loss_id_A + loss_id_B
        loss_G.backward()
        optimizer_G.step()

        optimizer_D_B.zero_grad()
        # Real
        pred_real_B = D_B(real_B)
        loss_D_real_B = criterion_GAN(pred_real_B, real_target_like(pred_real_B))
        # Fake
        fake_B_detached = fake_B_buffer.push_and_pop(fake_B.detach())
        pred_fake_B = D_B(fake_B_detached)
        loss_D_fake_B = criterion_GAN(pred_fake_B, fake_target_like(pred_fake_B))
        loss_D_B = (loss_D_real_B + loss_D_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        optimizer_D_A.zero_grad()
        pred_real_A = D_A(real_A)
        loss_D_real_A = criterion_GAN(pred_real_A, real_target_like(pred_real_A))
        fake_A_detached = fake_A_buffer.push_and_pop(fake_A.detach())
        pred_fake_A = D_A(fake_A_detached)
        loss_D_fake_A = criterion_GAN(pred_fake_A, fake_target_like(pred_fake_A))
        loss_D_A = (loss_D_real_A + loss_D_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # logging
        recon_cycle_loss += (loss_cycle_A.item() + loss_cycle_B.item())
        gan_loss_val += (loss_GAN_AB.item() + loss_GAN_BA.item())
        id_loss_val += (loss_id_A.item() + loss_id_B.item())

        if i % 50 == 0:
            loop.set_description(f"Epoch[{epoch}/{epochs}] step {i} lossG {loss_G.item():.4f}")

    # update lr
    scheduler_G.step()
    scheduler_D_A.step()
    scheduler_D_B.step()

    # Save samples & checkpoint
    if epoch % save_every == 0 or epoch == 1 or epoch == epochs:
        with torch.no_grad():
            G_AB.eval(); G_BA.eval()
            real_A_sample, _ = next(iter(loaderA))
            real_B_sample, _ = next(iter(loaderB))
            real_A_sample = real_A_sample.to(device) * 2.0 - 1.0
            real_B_sample = real_B_sample.to(device) * 2.0 - 1.0
            fake_B_sample = G_AB(real_A_sample).cpu() * 0.5 + 0.5
            fake_A_sample = G_BA(real_B_sample).cpu() * 0.5 + 0.5
            real_A_vis = (real_A_sample.cpu() * 0.5 + 0.5)
            real_B_vis = (real_B_sample.cpu() * 0.5 + 0.5)
            save_sample(epoch, 0, real_A_vis, real_B_vis, fake_B_sample, fake_A_sample, prefix="epoch")
        torch.save({
            'G_AB': G_AB.state_dict(),
            'G_BA': G_BA.state_dict(),
            'D_A': D_A.state_dict(),
            'D_B': D_B.state_dict(),
            'optimizer_G': optimizer_G.state_dict(),
            'optimizer_D_A': optimizer_D_A.state_dict(),
            'optimizer_D_B': optimizer_D_B.state_dict(),
        }, os.path.join(checkpoints_dir, f"cyclegan_epoch{epoch:03d}.pth"))
        print(f"[Epoch {epoch}] Saved samples and checkpoint.")


    print(f"Epoch {epoch} summary: cycle_loss {recon_cycle_loss/len(loaderA):.6f}, gan_loss {gan_loss_val/len(loaderA):.6f}, id_loss {id_loss_val/len(loaderA):.6f}")

elapsed = time.time() - start_time
print("Training finished in {:.1f} s".format(elapsed))


# Save final trained models
torch.save({
    'G_AB': G_AB.state_dict(),
    'G_BA': G_BA.state_dict(),
    'D_A': D_A.state_dict(),
    'D_B': D_B.state_dict(),
    'optimizer_G': optimizer_G.state_dict(),
    'optimizer_D_A': optimizer_D_A.state_dict(),
    'optimizer_D_B': optimizer_D_B.state_dict(),
    'epochs': epochs,
}, os.path.join(checkpoints_dir, "cyclegan_final.pth"))

print("Final model saved to:", os.path.join(checkpoints_dir, "cyclegan_final_T1.pth"))


# Harmonize Guys Images
print("Generating harmonized outputs for Site A ...")
G_AB.eval()
os.makedirs(out_dir, exist_ok=True)
with torch.no_grad():
    for i in range(len(dsA)):
        x, p = dsA[i]
        x_t = x.unsqueeze(0).to(device) * 2.0 - 1.0
        y = G_AB(x_t)              # in [-1,1]
        y = (y.cpu().squeeze(0).squeeze(0).numpy() * 0.5 + 0.5)  # to [0,1]
        y = (np.clip(y, 0.0, 1.0) * 255.0).astype(np.uint8)
        fname = os.path.basename(p)
        Image.fromarray(y).save(os.path.join(out_dir, fname))
print("Harmonized images saved to:", out_dir)



Num A: 88 Num B: 88 Device: cuda
Starting training...


Epoch[1/100] step 0 lossG 87.2736: 100%|██████████| 22/22 [00:33<00:00,  1.53s/it]


[Epoch 1] Saved samples and checkpoint.
Epoch 1 summary: cycle_loss 4.202847, gan_loss 2.825985, id_loss 19.107940


Epoch[2/100] step 0 lossG 16.4798: 100%|██████████| 22/22 [00:32<00:00,  1.47s/it]

Epoch 2 summary: cycle_loss 1.928274, gan_loss 0.751455, id_loss 8.750340


...

Epoch 98 summary: cycle_loss 0.516000, gan_loss 0.713071, id_loss 1.365697



Epoch[99/100] step 0 lossG 2.4914: 100%|██████████| 22/22 [00:34<00:00,  1.55s/it]

Epoch 99 summary: cycle_loss 0.513348, gan_loss 0.725563, id_loss 1.360714



Epoch[100/100] step 0 lossG 2.7165: 100%|██████████| 22/22 [00:34<00:00,  1.55s/it]


[Epoch 100] Saved samples and checkpoint.
Epoch 100 summary: cycle_loss 0.512170, gan_loss 0.717823, id_loss 1.357627
Training finished in 3446.1 s
Final model saved to: checkpoints/cyclegan/cyclegan_final_T1.pth
Generating harmonized outputs for Site A ...
Harmonized images saved to: /content/drive/MyDrive/finalT1T2Data/outputs/harmonized_Guys_train_T1


In [ ]:
!cp /content/checkpoints/cyclegan/cyclegan_final_T2.pth /content/drive/MyDrive/finalT1T2Data/samples/harmonizedData/cyclegan_final_T2.pth

In [ ]:
!cp /content/checkpoints/cyclegan/cyclegan_final_T1.pth /content/drive/MyDrive/finalT1T2Data/samples/harmonizedData/cyclegan_final_T1.pth

Generate T2 harmonized images for the testing data

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from glob import glob
from torchvision import transforms
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generator definition
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3),
            nn.InstanceNorm2d(dim),
        )

    def forward(self, x):
        return x + self.conv_block(x)

class ResnetGenerator(nn.Module):
    def __init__(self, input_nc=1, output_nc=1, ngf=64, n_blocks=6):
        super().__init__()
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, ngf, kernel_size=7),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(True),
        ]

        # Downsample
        mult = 1
        for _ in range(2):
            model += [
                nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(ngf * mult * 2),
                nn.ReLU(True),
            ]
            mult *= 2

        # ResNet blocks
        for _ in range(n_blocks):
            model += [ResnetBlock(ngf * mult)]

        # Upsample
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(ngf * mult, ngf * mult // 2,
                                   kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(ngf * mult // 2),
                nn.ReLU(True),
            ]
            mult //= 2

        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(ngf, output_nc, kernel_size=7),
            nn.Tanh(),
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


checkpoint_path = "/content/drive/MyDrive/finalT1T2Data/samples/harmonizedData/cyclegan_final_T2.pth"

G_AB = ResnetGenerator().to(device)
ckpt = torch.load(checkpoint_path, map_location=device)

G_AB.load_state_dict(ckpt["G_AB"])
G_AB.eval()


transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])



input_dir = "/content/drive/MyDrive/finalT1T2Data/T2_test_by_loc/Guys"
output_dir = "/content/drive/MyDrive/finalT1T2Data/harmonizedTest/T2"
os.makedirs(output_dir, exist_ok=True)

with torch.no_grad():
    for p in glob(os.path.join(input_dir, "*.png")):
        img = Image.open(p)
        x = transform(img).unsqueeze(0).to(device)   # [0,1]
        x = x * 2.0 - 1.0                             # → [-1,1]

        y = G_AB(x)                                   # → [-1,1]
        y = (y.squeeze().cpu().numpy() * 0.5 + 0.5)   # → [0,1]
        y = np.clip(y, 0, 1)

        y = (y * 255).astype(np.uint8)
        Image.fromarray(y).save(os.path.join(output_dir, os.path.basename(p)))


Generate T1 harmonized images for the testing data

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from glob import glob
from torchvision import transforms
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generator definition
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3),
            nn.InstanceNorm2d(dim),
        )

    def forward(self, x):
        return x + self.conv_block(x)

class ResnetGenerator(nn.Module):
    def __init__(self, input_nc=1, output_nc=1, ngf=64, n_blocks=6):
        super().__init__()
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, ngf, kernel_size=7),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(True),
        ]

        # Downsample
        mult = 1
        for _ in range(2):
            model += [
                nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(ngf * mult * 2),
                nn.ReLU(True),
            ]
            mult *= 2

        # ResNet blocks
        for _ in range(n_blocks):
            model += [ResnetBlock(ngf * mult)]

        # Upsample
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(ngf * mult, ngf * mult // 2,
                                   kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(ngf * mult // 2),
                nn.ReLU(True),
            ]
            mult //= 2

        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(ngf, output_nc, kernel_size=7),
            nn.Tanh(),
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


checkpoint_path = "/content/drive/MyDrive/finalT1T2Data/samples/harmonizedData/cyclegan_final_T1.pth"

G_AB = ResnetGenerator().to(device)
ckpt = torch.load(checkpoint_path, map_location=device)

G_AB.load_state_dict(ckpt["G_AB"])
G_AB.eval()


transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])



input_dir = "/content/drive/MyDrive/finalT1T2Data/T1_test_by_loc/Guys"
output_dir = "/content/drive/MyDrive/finalT1T2Data/harmonizedTest/T1"
os.makedirs(output_dir, exist_ok=True)

with torch.no_grad():
    for p in glob(os.path.join(input_dir, "*.png")):
        img = Image.open(p)
        x = transform(img).unsqueeze(0).to(device)   # [0,1]
        x = x * 2.0 - 1.0                             # → [-1,1]

        y = G_AB(x)                                   # → [-1,1]
        y = (y.squeeze().cpu().numpy() * 0.5 + 0.5)   # → [0,1]
        y = np.clip(y, 0, 1)

        y = (y * 255).astype(np.uint8)
        Image.fromarray(y).save(os.path.join(output_dir, os.path.basename(p)))
